In [1]:
%pip install pandas
%pip install openpyxl
%pip install nltk


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

data = pd.read_excel('hasil/hasil_preposesing.xlsx')

C:\Users\halim\AppData\Local\Temp\ipykernel_15368\522581935.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\halim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import csv

def read_lexicon(file_path):
    with open(file_path, 'r') as csvfile:
        lexicon = {row[0]: int(row[1]) for row in csv.reader(csvfile)}
    return lexicon

def sentiment_analysis_lexicon_indonesia(text, lexicon_positive, lexicon_negative):
    score = sum(lexicon_positive.get(word, 0) + lexicon_negative.get(word, 0) for word in text)
    if pd.isna(score):
        score = 0  # Set score to 0 if it's NaN
    unclassified_words = [word for word in text if word not in lexicon_positive and word not in lexicon_negative]
    return pd.Series([score, unclassified_words], index=['polarity_text', 'unclassified_words'])

lexicon_positive = read_lexicon('dataset/lexicon_positive.csv')
lexicon_negative = read_lexicon('dataset/lexicon_negative.csv')

def tokenize_and_analyze_sentiment(text, lexicon_positive, lexicon_negative):
    tokenized_text = word_tokenize(text)
    return sentiment_analysis_lexicon_indonesia(tokenized_text, lexicon_positive, lexicon_negative)


data[['polarity_text', 'unclassified_words']] = data['text_join'].fillna('').apply(lambda x: tokenize_and_analyze_sentiment(x, lexicon_positive, lexicon_negative))
data

,text,emoticon,rating,polarity_emotion,cf,clean,token,normalisasi,stopwords,stemming,text_join,polarity_text,unclassified_words
0,Aplikasinya bagus semua nya bagus tapi tolong ...,"['🏻', '👍']",5,1,aplikasinya bagus semua nya bagus tapi tolong ...,aplikasinya bagus semua nya bagus tapi tolong ...,"['aplikasinya', 'bagus', 'semua', 'nya', 'bagu...","['aplikasinya', 'bagus', 'semua', 'nya', 'bagu...","['aplikasinya', 'bagus', 'nya', 'bagus', 'tolo...","['aplikasi', 'bagus', 'nya', 'bagus', 'tolong'...",aplikasi bagus nya bagus tolong tambah teman a...,-16,"[nya, teman, nya]"
1,Lewat aplikasi ruang guru saya bisa tau pembel...,[],5,0,lewat aplikasi ruang guru saya bisa tau pembel...,lewat aplikasi ruang guru saya bisa tau pembel...,"['lewat', 'aplikasi', 'ruang', 'guru', 'saya',...","['lewat', 'aplikasi', 'ruang', 'guru', 'saya',...","['aplikasi', 'ruang', 'guru', 'pembelajaran', ...","['aplikasi', 'ruang', 'guru', 'ajar', 'pembela...",aplikasi ruang guru ajar pembelajaranya erti,-6,[pembelajaranya]
2,Seru isinya bisa ngerjain tugas dr aplikasi at...,[],5,0,seru isinya bisa ngerjain tugas dr aplikasi at...,seru isinya bisa ngerjain tugas dr aplikasi at...,"['seru', 'isinya', 'bisa', 'ngerjain', 'tugas'...","['seru', 'isinya', 'bisa', 'mengerjakan', 'tug...","['seru', 'isinya', 'tugas', 'aplikasi', 'guru'...","['seru', 'isi', 'tugas', 'aplikasi', 'guru', '...",seru isi tugas aplikasi guru ngajar,-5,[ngajar]
3,Baik,[],5,0,baik,baik,['baik'],['baik'],[],[],NaN,0,[]
4,Jgn di dwnld Nanti bayar,[],3,0,jgn di dwnld nanti bayar,jgn di dwnld nanti bayar,"['jgn', 'di', 'dwnld', 'nanti', 'bayar']","['jangan', 'di', 'dwnld', 'nanti', 'bayar']","['dwnld', 'bayar']","['dwnld', 'bayar']",dwnld bayar,-2,[dwnld]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13939,Keren,[],5,0,keren,keren,['keren'],['keren'],['keren'],['keren'],keren,0,[keren]
13940,gw ngga ngerti cara mainnya,[],2,0,gw ngga ngerti cara mainnya,gw ngga ngerti cara mainnya,"['gw', 'ngga', 'ngerti', 'cara', 'mainnya']","['saya', 'tidak', 'mengerti', 'cara', 'mainnya']","['mengerti', 'mainnya']","['erti', 'main']",erti main,0,[]
13941,NaN,"['🏻', '🤝']",5,0,NaN,NaN,['nan'],['nan'],['nan'],['nan'],NaN,0,[]
13942,Paket 3 bulan Explorers hanya masuk 3x mengapa...,[],1,0,paket 3 bulan explorers hanya masuk 3x mengapa...,paket bulan explorers hanya masuk mengapa dil...,"['paket', 'bulan', 'explorers', 'hanya', 'masu...","['paket', 'bulan', 'explorers', 'hanya', 'masu...","['paket', 'explorers', 'masuk', 'diluar', 'dug...","['paket', 'explorers', 'masuk', 'luar', 'duga']",paket explorers masuk luar duga,6,"[explorers, masuk, luar]"


In [4]:
unclassified_words = set()

for value in data['unclassified_words']:
    if isinstance(value, list) and value:
        unclassified_words.update(value)

filtered_df = pd.DataFrame(list(unclassified_words), columns=['unclassified_words'])

#simpan data emoticon yang tidak bisa klasifikasi
filtered_df.to_excel('gagal_proses/unclassified_words.xlsx', index=False)

#hapus kolom unprocessable_emoticons tidak butuh
data.drop('unclassified_words', axis=1, inplace=True)

In [5]:
data.to_excel('hasil/hasil_preposesing.xlsx', index=False)